# Dependencies

In [25]:
import os
import requests

from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

from transformers import ViltProcessor, ViltForQuestionAnswering, Blip2Processor, Blip2Model, Blip2ForConditionalGeneration

from word2number import w2n
import ot


# Using Blip2 for image captioning(optional)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16
) 


image_folder = "data/shape1/"
output_file = "data/shape1.txt"


image_files = [img for img in os.listdir(image_folder) if img.endswith((".png", ".jpg", ".jpeg"))]
image_files.sort(key=lambda x: int(x.split('.')[0]))


with open(output_file, "w") as f:
    for image_name in image_files:
        image_path = os.path.join(image_folder, image_name)
        image = Image.open(image_path)
        
     
        inputs = processor(images=image, return_tensors="pt", padding=True).to(device, torch.float16)

        generated_ids = model.generate(**inputs)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip() 
    
        f.write(f"{image_name}: {generated_text}\n")

# Helper functions

In [26]:
class ConsistencyClassifier(nn.Module):
    def __init__(self, hidden_size):
        super(ConsistencyClassifier, self).__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.dense(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        x = self.sigmoid(x)
        return x


def sinkhorn_distance(x, y, epsilon=1.0, num_iters=20):
    """

    Args:
        x (torch.Tensor): Tensor of shape (batch_size, num_features_x, feature_dim)
        y (torch.Tensor): Tensor of shape (batch_size, num_features_y, feature_dim)
        epsilon (float): 
        num_iters (int): Sinkhorn 

    Returns:
        torch.Tensor:
    """
    x_col = x.unsqueeze(2)  # (batch_size, num_features_x, 1, feature_dim)
    y_lin = y.unsqueeze(1)  # (batch_size, 1, num_features_y, feature_dim)
    C = torch.sum((x_col - y_lin) ** 2, dim=-1)  # (batch_size, num_features_x, num_features_y) 32 9 144
    
    C = C / C.max()
    #  e^(-C/epsilon) 32 9 144
    K = torch.exp(-C / epsilon)
    log_K = -C / epsilon

    # a, b means
    a = torch.ones(x.size(0), x.size(1), device=x.device) / x.size(1)
    b = torch.ones(y.size(0), y.size(1), device=y.device) / y.size(1)

    # Sinkhorn 
    log_u = torch.zeros_like(a)
    log_v = torch.zeros_like(b)
    for _ in range(num_iters):
        log_u = torch.log(a) - torch.logsumexp(log_K + log_v.unsqueeze(1), dim=2)
        log_v = torch.log(b) - torch.logsumexp(log_K.transpose(-2, -1) + log_u.unsqueeze(1), dim=2)


    log_T = log_u.unsqueeze(-1) + log_K + log_v.unsqueeze(-2)
    T = torch.exp(log_T) 


    distance = torch.sum(T * C, dim=(-2, -1))

    return distance


def batch_loader(file_path, batch_size=32):
    batch_images = []
    batch_questions = []
    batch_labels = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(',')
            if len(parts) == 4:
                question = parts[1]
                answer = parts[2]
                image_path = parts[3]

                try:
                    img = Image.open(image_path).resize((384, 384))
                    batch_images.append(img)
                    batch_questions.append(question)
                    ans1 = w2n.word_to_num(answer)
                    batch_labels.append(str(ans1))

                    if len(batch_images) == batch_size:
                        yield batch_images, batch_questions, batch_labels
                        batch_images, batch_questions, batch_labels = [], [], []
                except IOError:
                    print(f"Error opening image {image_path}")
        if batch_images:  # Yield any remaining data as the last batch
            yield batch_images, batch_questions, batch_labels


def process_and_predict(images, questions, processor, model):

    encoding = processor(images, questions, return_tensors="pt", padding=True)
    outputs = model(**encoding)
    
def valid(model, best_acc, processor):
    print("--------------------this is validation------------------------")
    accuracy = []
    model.cpu()
    model.eval()
    for images, questions, labels in batch_loader('data/val_data.txt'):#ata/val_data
        outputs = process_and_predict(images, questions, processor, model)
        print(outputs.logits)
        probabilities = torch.softmax(outputs.logits, dim=1)
        max_prob_indices = torch.argmax(probabilities, dim=1)
        word_list = []
        for i,k in  enumerate(max_prob_indices):
            print(questions[i])
            print("-------------ground truth --------------")
            print(labels[i])
            print("-------------our answer --------------")
            print(model.config.id2label[int(k)])
            word_list.append(model.config.id2label[int(k)])
        matches = sum(1 for x, y in zip(word_list, labels) if x == y)
        probability = matches / len(labels)
        print("-------------batch accuracy --------------")
        accuracy.append(probability)
        print(probability)

    average = sum(accuracy) / len(accuracy)
    print("-------------average accuracy --------------")
    print(average)
    if average> best_acc:
        best_acc = average 
        model.save_pretrained('checkpoint/')
        print("save in:{save_path}")

# Main train loop(val)

for demo purpose, we don't show the whole training process on our own laptop

In [ ]:
processor = ViltProcessor.from_pretrained("./checkpoint")
model = ViltForQuestionAnswering.from_pretrained("./checkpoint").cuda()
loss_function = torch.nn.CrossEntropyLoss()
criteria = nn.BCELoss()
cls = ConsistencyClassifier(hidden_size=768)
optimizer = torch.optim.Adam(
    list(cls.parameters()) + list(model.parameters()), lr=0.0001
)
scheduler = CosineAnnealingLR(optimizer, T_max=15, eta_min=0)

beta = 0.001
theta = 0.000005


for epoch in range(15):
    model.cuda()
    cls.cuda()
    cls.train()
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    avg_train_loss = []
    avg_train_acc = []
    num = 0

    for images, questions, labels in batch_loader(
        "data/train_data.txt", batch_size=32
    ):

        # image_list = [transforms.ToTensor()(image) for image in images]
        label_list = [model.config.label2id[label] for label in labels]
        labels_tensor = torch.tensor(label_list).cuda()

        encoding = processor(
            images, questions, return_tensors="pt", padding=True, truncation=True
        )
        for feature, data in encoding.data.items():
            encoding.data[feature] = data.cuda()

        # outputs = model(**encoding)

        outputs = model.vilt(**encoding)
        text_feature = outputs.last_hidden_state[:, 1:10, :]
        image_feature = outputs.last_hidden_state[:, -144:, :]

        # SinkhornDistance
        normalized_text_features = text_feature / text_feature.sum(dim=1, keepdim=True)
        normalized_image_features = image_feature / image_feature.sum(
            dim=1, keepdim=True
        )

        #
        OT_distance = sinkhorn_distance(
            normalized_text_features, normalized_image_features
        )
        loss2 = OT_distance.mean()

        logits = model.classifier(outputs.pooler_output)
        labels = torch.FloatTensor([[1] for i in range(len(labels))])
        loss1 = criteria(cls(outputs.pooler_output), labels.cuda())

        # logits = outputs.logits
        optimizer.zero_grad()
        loss = (
            loss_function(logits, labels_tensor)
            + beta * loss1
            + theta * loss2
        )
        loss.backward()
        optimizer.step()
        num += len(labels)

        train_loss += loss.item() * len(labels)
        ret, predictions = torch.max(logits.data, 1)
        correct_counts = predictions.eq(labels_tensor.data.view_as(predictions))
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
        train_acc += acc.item() * len(labels)

        avg_train_loss.append(train_loss)
        avg_train_acc.append(train_acc)

    avg_train_loss1 = sum(avg_train_loss) / num
    avg_train_acc1 = sum(avg_train_acc) / num
    print(
        f"Epoch {epoch+1}, Average Training Loss: {avg_train_loss1}, Average Training Accuracy: {avg_train_acc1}"
    )

    scheduler.step()
    best_acc = 0
    valid(model, best_acc, processor)

# Test on the testset

In [ ]:
accuracy = []

for images, questions, labels in batch_loader('data/test_data.txt'):
    outputs = process_and_predict(images, questions, processor, model)
    print(outputs.logits)

    probabilities = torch.softmax(outputs.logits, dim=1)
    max_prob_indices = torch.argmax(probabilities, dim=1)
    word_list = []

    for i,k in  enumerate(max_prob_indices):
        print(questions[i])
        print("-------------ground truth --------------")
        print(labels[i])
        print("-------------our answer --------------")
        print(model.config.id2label[int(k)])
        word_list.append(model.config.id2label[int(k)])

    matches = sum(1 for x, y in zip(word_list, labels) if x == y)
    probability = matches / len(labels)
    print("-------------batch accuracy --------------")
    accuracy.append(probability)
    print(probability)

average = sum(accuracy) / len(accuracy)
print("-------------average accuracy --------------")
print(average)

# Examples

In [ ]:
processor = ViltProcessor.from_pretrained("./checkpoint")
model = ViltForQuestionAnswering.from_pretrained("./checkpoint").cuda()


def predict(image_path, question, processor, model):
    
    device = "cuda" if torch.cuda.is_available() else "cpu"

    image = Image.open(image_path).resize((384, 384)) 

    encoding = processor(image, question, return_tensors="pt", padding=True)
    
    # Move the encoded tensors to the same device as the model
    for key in encoding:
        encoding[key] = encoding[key].to(device)

    # Move the model to the device
    model = model.to(device)

    # Generate outputs using the model
    output = model(**encoding)

    prob = torch.softmax(output.logits, dim=1)
    max_prob_indice = torch.argmax(prob, dim=1)
    word_list = []

    print(question)

    for i,k in  enumerate(max_prob_indice):

        print("-------------our answer --------------")
        print(model.config.id2label[int(k)])
        word_list.append(model.config.id2label[int(k)])


In [ ]:
image_path = "data/shape/10.png"
question = "how many green squares are in the image?"

predict(image_path,question,processor,model)